<a href="https://colab.research.google.com/github/Sangyeonglee353/ai-musthave/blob/main/Chapter_05_%EC%9C%A0%ED%96%89_%EB%94%B0%EB%9D%BC%EA%B0%80%EA%B8%B0_ResNet_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.3 기본 블록 정의하기

In [1]:
# ResNet 기본 블록
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size=3):
    super(BasicBlock, self).__init()

    # 1. 합성곱층 정의
    self.c1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1)
    self.c2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=1)
    self.downsample = nn.Conv2d(in_channles, out_channels, kernel_size=1)

    # 2. 배치 정규화층 정의
    self.bn1 = nn.BatchNorm2d(num_features=out_channels)
    self.bn2 = nn.BatchNorm2d(num_features=out_channels)

    self.relu = nn.ReLU()

In [2]:
# 기본 블록의 순전파 정의
def forward(self, x):
  # 3. 스킵 커넥션을 위해 초기 입력 저장
  x_ = x

  # ResNet 기본 블록에서 F(x) 부분
  x = self.c1(x)
  x = self.bn1(x)
  x = self.relu(x)
  x = self.c2(x)
  x = self.bn2(x)

  # 4. 합성곱층의 결과와 입력의 채널 수를 맞춤
  x_ = self.downsample(x_)

  # 5. 합성곱층의 결과와 저장해놨던 입력값을 더해줌(스킵 커넥션)
  x += x_
  x = self.relu(x)

  return x

# 5.4 ResNet 모델 정의하기

In [3]:
# ResNet 모델 정의하기
class ResNet(nn.Module):
  def __init__(self, num_classes=10):
    super(Resnet, self).__init__()

    # 1. 기본 블록
    self.b1 = BasicBlock(in_channels=3, out_channels=64)
    self.b2 = BasicBlock(in_channels=64, out_channels=128)
    self.b3 = BasicBlock(in_channels=128, out_channels=256)

    # 2. 풀링을 최댓값이 아닌 평균값으로
    self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    # 3. 분류기
    self.fc1 = nn.Linear(in_features=4096, out_features=2048)
    self.fc2 = nn.Linear(in_features=2048, out_features=512)
    self.fc3 = nn.Linear(in_features=512, out_features=num_classes)

    self.relu = nn.ReLU()

In [4]:
# ResNet의 순전파 정의
def forward(self, x):
  # 1. 기본 블록과 풀링층 통과
  x = self.b1(x)
  x = self.pool(x)
  x = self.b2(x)
  x = self.pool(x)
  x = self.b3(x)
  x = self.pool(x)

  # 2. 분류기의 입력으로 사용하기 위한 평탄화
  x = torch.flatten(x, start_dim=1)

  # 3. 분류기로 예측값 출력
  x = self.fc1(x)
  x = self.relu(x)
  x = self.fc2(x)
  x = self.relu(x)
  x = self.fc3(x)

  return x

# 5.5 모델 학습하기